In [68]:
import numpy as np
import pandas as pd
np.set_printoptions(threshold=np.nan)
from sklearn.preprocessing import scale

In [11]:
signal_data = pd.read_csv("/home/ywu/Documents/workplace/AFP_data/rps_signals.csv")
signal_data = signal_data.set_index("DATE")
#signal_data.dropna(how = "any", axis = 1 ,inplace = True)
signal_data.head()

,permno,gvkey,fyear,sic2,spi,mve_f,bm,ep,cashpr,dy,...,baspread,std_dolvol,std_turn,ill,zerotrade,BETA,betasq,rsq1,pricedelay,idiovol
DATE,,,,,,,,,,,,,,,,,,,,,
19800131,10006,1010,1978,37,0.0000,269.308500,1.180962,0.153022,-32.218678,0.067347,...,0.014234,0.881844,0.635898,2.565667e-08,1.115306e-07,1.060404,1.124456,0.343403,0.029861,0.025576
19800131,10057,1098,1978,36,0.0000,97.372000,0.956692,0.135131,-4.408581,0.036982,...,0.022470,1.368363,2.546787,2.719812e-07,6.199128e-08,1.525993,2.328654,0.307902,0.092675,0.037473
19800131,10103,1012,1978,33,NaN,1.697500,3.362003,0.338144,-17.143817,0.000000,...,0.511667,NaN,NaN,NaN,NaN,1.759501,3.095842,0.096756,0.221852,0.087020
19800131,10137,1279,1978,49,NaN,537.524500,1.330341,0.153238,-87.819837,0.137560,...,0.020899,0.553246,0.740017,1.765620e-08,9.726790e-08,0.492885,0.242936,0.189697,0.125795,0.017540
19800131,10145,1300,1978,99,-0.0031,805.633282,1.579284,0.149248,-22.050470,0.070247,...,0.016947,0.427617,0.657563,2.898901e-09,6.190654e-08,1.139150,1.297663,0.279436,0.024234,0.031201


In [4]:
fama_french = pd.read_csv("/home/ywu/Documents/workplace/AFP_data/F-F_Research_Data_5_Factors_2x3_daily.CSV", skiprows = 3)
fama_french.set_index("DATE", inplace = True)
fama_french.head()

,Mkt-RF,SMB,HML,RMW,CMA,RF
DATE,,,,,,
19630701,-0.67,0.00,-0.32,0.01,0.15,0.012
19630702,0.79,-0.27,0.27,-0.08,-0.19,0.012
19630703,0.63,-0.17,-0.09,0.19,-0.33,0.012
19630705,0.40,0.08,-0.28,0.07,-0.33,0.012
19630708,-0.63,0.04,-0.17,-0.31,0.13,0.012


In [5]:
def get_signal_names():
    names = [ 'absacc',
              'acc',
              'aeavol',
              'age',
              'baspread',
              'beta',
              'betasq',
              'bm',
              'bm_ia',
              'cash',
              'cashdebt',
              'cashpr',
              'cfp',
              'cfp_ia',
              'chatoia',
              'chcsho',
              'chempia',
              'chfeps',
              'chinv',
              'chmom',
              'chnanalyst',
              'chpmia',
              'chtx',
              'cinvest',
              'convind',
              'currat',
              'depr',
              'disp',
              'divi',
              'orgcap',
              'pchcapx_ia',
              'pchcurrat',
              'pchdepr',
              'pchgm_pchsale',
              'pchquick',
              'pchsale_pchinvt',
              'pchsale_pchxsga',
              'pchsaleinv',
              'pctacc',
              'pricedelay',
              'ps',
              'divo',
              'dolvol',
              'dy',
              'ear',
              'egr',
              'ep',
              'fgr5yr',
              'gma',
              'grcapx',
              'grltnoa',
              'herf',
              'hire',
              'idiovol',
              'ill',
              'indmom',
              'invest',
              'ipo',
              'lev',
              'lgr',
              'maxret',
              'mom12m',
              'mom1m',
              'mom36m',
              'mom6m',
              'ms',
              'mve',
              'mve_ia',
              'nanalyst',
              'nincr',
              'operprof',
              'roeq',
              'roic',
              'rsup',
              'salecash',
              'saleinv',
              'salerec',
              'secured',
              'securedind',
              'sfe',
              'sgr',
              'sin',
              'sp',
              'std_dolvol',
              'quick',
              'rd',
              'rd_mve',
              'rd_sale',
              'realestate',
              'retvol',
              'roaq',
              'roavol',
              'std_turn',
              'stdacc',
              'stdcf',
              'sue',
              'tang',
              'tb',
              'turn',
              'zerotrade'
            ]
            
    return names

In [10]:
fama_french_cleaned = fama_french.loc[signal_data.index]
fama_french_cleaned = fama_french_cleaned[~fama_french_cleaned .index.duplicated(keep='first')]
fama_french_cleaned.head()

,Mkt-RF,SMB,HML,RMW,CMA,RF
DATE,,,,,,
19800131,-0.72,0.57,0.17,-0.04,0.11,0.036
19800229,0.97,-0.68,-0.11,-0.22,-0.15,0.044
19800331,1.46,0.13,-0.16,-0.19,-0.14,0.057
19800430,0.29,0.00,0.14,-0.09,0.20,0.059
19800530,0.70,-0.39,0.02,-0.03,0.01,0.038


In [47]:
def divide_data(dataset):
    gb = dataset.groupby(dataset.index)
    for x in gb.groups:
        gb.get_group(x).to_csv(str(x)+".csv")

In [74]:
def clean_data(dataset):
    gb = dataset.groupby(dataset.index)
    for x in gb.groups:
        temp = gb.get_group(x)
        permno = temp["permno"]
        gvkey = temp["gvkey"]
        temp = temp.loc[:,get_signal_names()]
        temp = temp.rank()
        temp = temp.fillna(0)
        scaled =  scale(temp)
        result = pd.DataFrame(scaled, index = temp.index, columns = temp.columns)
        #dataset = normalize(dataset)
        result["permno"] = permno
        result["gvkey"] = gvkey
        result.to_csv("Data2/"+str(x)+".csv")

In [66]:
def get_ticker(dataset, num = 1000, col = "mve"):
    return dataset.nlargest(num, col)

In [20]:
data1 = pd.read_csv("/home/ywu/Documents/Github/afp/data_cleaning/Data/19800131.csv")
data1.head()

,DATE,permno,gvkey,fyear,sic2,mve_f,bm,ep,age,spii,...,count,ewret,mom1m,IPO,Mkt-RF,SMB,HML,RMW,CMA,RF
0,19800131,10006,1010,1978,37,269.308500,1.180962,0.153022,4,0,...,44,0.097817,0.000000,0,-0.72,0.57,0.17,-0.04,0.11,0.036
1,19800131,10057,1098,1978,36,97.372000,0.956692,0.135131,4,0,...,46,0.097817,0.174157,0,-0.72,0.57,0.17,-0.04,0.11,0.036
2,19800131,10103,1012,1978,33,1.697500,3.362003,0.338144,1,0,...,10,0.097817,0.500000,1,-0.72,0.57,0.17,-0.04,0.11,0.036
3,19800131,10137,1279,1978,49,537.524500,1.330341,0.153238,4,0,...,44,0.097817,-0.047761,0,-0.72,0.57,0.17,-0.04,0.11,0.036
4,19800131,10145,1300,1978,99,805.633282,1.579284,0.149248,4,1,...,43,0.097817,0.085635,0,-0.72,0.57,0.17,-0.04,0.11,0.036


In [75]:
data2 = clean_data(signal_data)